In [3]:
import sage_data_client
import matplotlib.pyplot as plt
import pandas as pd
from metpy.calc import dewpoint_from_relative_humidity, wet_bulb_temperature
from metpy.units import units
from PIL import Image
import numpy as np
import datetime
import xarray as xr
import os

from matplotlib.dates import DateFormatter

In [ ]:
wxt_global_NEIU = {'conventions': "CF 1.10",
                   'site_ID' : "NEIU",
                  'CAMS_tag' : "CMS-WXT-002",
                  'datastream' : "CMS_wxt536_NEIU_a1",
                  'datalevel' : "a1",
                  'latitude' : 41.9804526,
                  'longitude' : -87.7196038}

var_attrs_aqt = {'temperature': {'standard_name' : 'air_temperature',
                       'units' : 'celsius'},
                'humidity': {'standard_name' : 'relative_humidity',
                       'units' : 'percent'},
                'dewpoint': {'standard_name' : 'dew_point_temperature',
                       'units' : 'celsius'},
                'pressure': {'standard_name' : 'air_pressure',
                       'units' : 'hPa'},
                'wetbulb': {'standard_name' : 'wet_bulb_temperature',
                       'units' : 'celsius'},
                'wetbulb': {'standard_name' : 'wet_bulb_temperature',
                       'units' : 'celsius'},
                'wind_dir_10s': {'standard_name' : 'wet_bulb_temperature',
                       'units' : 'celsius'}}

In [4]:
def ingest_wxt(st):
    hours = 24
    start = st.strftime('%Y-%m-%dT%H:%M:%SZ')
    end = (st + datetime.timedelta(hours=hours)).strftime('%Y-%m-%dT%H:%M:%SZ')
    print(start)
    print(end)
    df_temp = sage_data_client.query(start=start,
                                     end=end, 
                                        filter={
                                            "name" : 'wxt.env.temp|wxt.env.humidity|wxt.env.pressure|wxt.rain.accumulation',
                                            "plugin" : "registry.sagecontinuum.org/jrobrien/waggle-wxt536:0.23.5.*",
                                            "vsn" : "W08D",
                                            "sensor" : "vaisala-wxt536"
                                        }
    )
    winds = sage_data_client.query(start=start,
                                     end=end, 
                                        filter={
                                            "name" : 'wxt.wind.speed|wxt.wind.direction',
                                            "plugin" : "registry.sagecontinuum.org/jrobrien/waggle-wxt536:0.23.5.*",
                                            "vsn" : "W08D",
                                            "sensor" : "vaisala-wxt536"
                                        }
    )
    
    hums = df_temp[df_temp['name']=='wxt.env.humidity']
    temps = df_temp[df_temp['name']=='wxt.env.temp']
    pres = df_temp[df_temp['name']=='wxt.env.pressure']
    rain = df_temp[df_temp['name']=='wxt.rain.accumulation']


    npres = len(pres)
    nhum = len(hums)
    ntemps = len(temps)
    nrains = len(rain)
    print(npres, nhum, ntemps, nrains)
    minsamps = min([nhum, ntemps, npres, nrains])

    temps['time'] = pd.DatetimeIndex(temps['timestamp'].values)

    vals = temps.set_index('time')[0:minsamps]
    vals['temperature'] = vals.value.to_numpy()[0:minsamps]
    vals['humidity'] = hums.value.to_numpy()[0:minsamps]
    vals['pressure'] = pres.value.to_numpy()[0:minsamps]
    vals['rainfall'] = rain.value.to_numpy()[0:minsamps]

    direction = winds[winds['name']=='wxt.wind.direction']
    speed = winds[winds['name']=='wxt.wind.speed']

    nspeed = len(speed)
    ndir = len(direction)
    print(nspeed, ndir)
    minsamps = min([nspeed, ndir])

    speed['time'] = pd.DatetimeIndex(speed['timestamp'].values)
    windy = speed.set_index('time')[0:minsamps]
    windy['speed'] = windy.value.to_numpy()[0:minsamps]
    windy['direction'] = direction.value.to_numpy()[0:minsamps]


    winds10mean = windy.resample('10S').mean(numeric_only=True).ffill()
    winds10max = windy.resample('10S').max(numeric_only=True).ffill()
    dp = dewpoint_from_relative_humidity( vals.temperatures.to_numpy() * units.degC, 
                                         vals.humidity.to_numpy() * units.percent)

    vals['dewpoint'] = dp
    vals10 = vals.resample('10S').mean(numeric_only=True).ffill() #ffil gets rid of nans due to empty resample periods
    wb = wet_bulb_temperature(vals10.pressure.to_numpy() * units.hPa,
                              vals10.temperatures.to_numpy() * units.degC,
                              vals10.dewpoint.to_numpy() * units.degC)

    vals10['wetbulb'] = wb
    vals10['wind_dir_10s'] = winds10mean['direction']
    vals10['wind_mean_10s'] = winds10mean['speed']
    vals10['wind_max_10s'] = winds10max['speed']
    _ = vals10.pop('value')
    fname = st.strftime('/Users/scollis/data/wxt/CMS_wxt536_NEIU_a1_%Y%m%d_%H%M%SZ.nc')
    
    try:
        os.remove(fname)
    except OSError:
        pass
    
    vals10xr = xr.Dataset.from_dataframe(vals10)
    vals10xr = vals10xr.sortby('time')
    
    vals10xr.to_netcdf(fname)



In [ ]:
ingest_wxt(datetime.datetime(2023,6,15))


In [5]:
start_date = datetime.datetime(2023,5,5)
for i in range(45):
    this_date = start_date + datetime.timedelta(days=i)
    print(this_date)
    try:
        ingest_wxt(this_date)
        print("Succeed")
    except:
        print("Fail")

2023-05-05 00:00:00
2023-05-05T00:00:00Z
2023-05-06T00:00:00Z
316497 316497 316497 316497


/var/folders/8_/6l9wr7s54bs85ldw43g4btbm0000gq/T/ipykernel_7334/3121474618.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temps['time'] = pd.DatetimeIndex(temps['timestamp'].values)


316497 316497


/var/folders/8_/6l9wr7s54bs85ldw43g4btbm0000gq/T/ipykernel_7334/3121474618.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  speed['time'] = pd.DatetimeIndex(speed['timestamp'].values)


Succeed
2023-05-06 00:00:00
2023-05-06T00:00:00Z
2023-05-07T00:00:00Z
901594 901594 901594 901594


...

Succeed
2023-06-15 00:00:00
2023-06-15T00:00:00Z
2023-06-16T00:00:00Z
751970 751856 752096 752224


/var/folders/8_/6l9wr7s54bs85ldw43g4btbm0000gq/T/ipykernel_7334/3121474618.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temps['time'] = pd.DatetimeIndex(temps['timestamp'].values)


752765 752640


/var/folders/8_/6l9wr7s54bs85ldw43g4btbm0000gq/T/ipykernel_7334/3121474618.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  speed['time'] = pd.DatetimeIndex(speed['timestamp'].values)


Succeed
2023-06-16 00:00:00
2023-06-16T00:00:00Z
2023-06-17T00:00:00Z
0 0 0 0
0 0
Fail
2023-06-17 00:00:00
2023-06-17T00:00:00Z
2023-06-18T00:00:00Z
0 0 0 0
0 0
Fail
2023-06-18 00:00:00
2023-06-18T00:00:00Z
2023-06-19T00:00:00Z
0 0 0 0
0 0
Fail


In [ ]:
ingest_wxt(datetime.datetime(2023,6,12))
ingest_wxt(datetime.datetime(2023,6,13))


In [ ]:
#ingest_wxt(datetime.datetime(2023,6,7))
#ingest_wxt(datetime.datetime(2023,6,1))
ingest_wxt(datetime.datetime(2023,6,2))




In [ ]:
! rm /Users/scollis/data/wxt/CMS_wxt536_NEIU_a1_20230612_000000Z.nc
